# Sistemas Lineares, Erros e Aritmética de Pontos Flutuantes


Nesse notebook será desenvolvido o projeto que irá responder os desafios propostos na Atividade 01 de MS211 - Cálculo Numérico

## Instalação de Bibliotecas


In [1]:
%pip install numpy
%pip install scipy

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


## Importando bibliotecas

In [2]:
import numpy as np
import scipy

### Questão 1

Considerando o sistema linear $Ax = b$, em que
$$A = \begin{bmatrix} 0.1 & 0.2 & 0.3 \\ 0.4 & 0.5 & 0.6 \\ 0.7 & 0.8 & 0.9 \end{bmatrix}, \quad b = \begin{bmatrix} 0.1 \\ 0.3 \\ 0.5 \end{bmatrix}$$

In [3]:
A = np.array([[0.1, 0.2, 0.3], 
              [0.4, 0.5, 0.6], 
              [0.7, 0.8, 0.9]])
b = np.array([[0.1],[0.3],[0.5]])

Uma matriz é dita singular quando ela não possui inversa, isso também pode ser interpretado como o determinante dessa matriz sendo igual a 0. <br>
Abaixo, vemos que o determinante é muito próximo de 0.

In [4]:
det_A = np.linalg.det(A)
det_A

6.661338147750926e-18

#### Item A)

Como o cálculo do determinante de uma matriz envolve produtos de pivôs, primeiro vamos alterar essa matriz para que não aconteçam erros devido a pontos flutuantes.<br>
Sabe-se que se uma matriz B puder ser escrita como B = $k^n$*A, com k sendo escalar, n sendo a dimensão da matriz e A uma matriz de mesma dimensão que B, então o determinante da matriz B será

$$ det(B) = k^n*det(A) $$

Vamos dizer que a matriz B auxiliar será B = 10*A


In [5]:
B = 10*A
B

array([[1., 2., 3.],
       [4., 5., 6.],
       [7., 8., 9.]])

Daí então, calcula-se o determinante da matriz B

In [6]:
det_B = np.linalg.det(B)
det_B

0.0

Ou seja, det(B) = 0 = $10^3$*det(A), logo det(A) = 0. <br>
Como a matriz A é singular, o sistema pode ser inconsistente, ou consistente com infinitas soluções. Para analisar isso, vemos o posto das duas matrizes.

In [7]:
rank_A = np.linalg.matrix_rank(A)
Ab = np.hstack((A, b.reshape(-1, 1)))
rank_Ab = np.linalg.matrix_rank(Ab)
if rank_Ab == rank_A:
    print("Sistema Consistente")
else:
    print("Sistema Inconsistente")

Sistema Consistente


Como o posto de A é igual ao posto da matriz aumentada $A | b$, então o sistema é consistente, logo possui infinitas soluções.<br>
O conjunto solução é do tipo 
$$ x = x_p + t v, \quad t \in \mathbb{R} $$

In [24]:
sol_particular, _, _, _ = np.linalg.lstsq(B, b, rcond=None)
nucleo = scipy.linalg.null_space(B)
sol_particular, nucleo

(array([[0.03888889],
        [0.02222222],
        [0.00555556]]),
 array([[-0.40824829],
        [ 0.81649658],
        [-0.40824829]]))

Nesse caso "sol_particular" é o $xp$ e "nucleo" é a base ortonormal $v$ para o espaço A

#### Item B)


Com aritmética exata, o método falha quando o último pivô é 0.<br>
Vamos verificar os pivôs da decomposição LU

In [9]:
P, L, U = scipy.linalg.lu(A)
np.diag(U)

array([7.00000000e-01, 8.57142857e-02, 1.11022302e-16])

Fazendo eliminação direta em aritmética exata sobre a matriz inteira 𝐵 (proporcional a 𝐴): <br>
Pivot $𝑎_{11} = 1$. Eliminar abaixo:<br>
$𝑅_2 ← 𝑅_2 − 4*𝑅_1 ⇒ [0, −3, −6]$<br>
$𝑅_3 ← 𝑅_3 − 7*𝑅_1 ⇒ [0, −6, −12]$<br>
Pivot na segunda etapa é −3 (não zero). Eliminar abaixo:<br>
$𝑅_3 ← 𝑅_3 −2*𝑅_2 ⇒ [0, 0, 0]$

Agora restou uma linha zero; ao tentar usar essa linha como terceiro pivot, ele seria 0 — não há pivô e o processo não consegue obter uma matriz triangular com pivôs não-nulos para voltar a uma solução única. Portanto não é possível avançar na eliminação ao tentar obter o terceiro pivô (o sistema é singular), o que indica número infinito de soluções (ou inconsistência, se a linha zero viesse acompanhada de termo independente não-zero).<br>

Com pivoteamento parcial não se evita essa situação porque as linhas são proporcionais de modo que o terceiro pivô será zero: não há um elemento não nulo útil para o terceiro pivô.

#### Item C)

Em representações de ponto-flutuante (base 2), números como 0.1 não são exatamente representáveis. Portanto a matriz 𝐴 armazenada no computador não é exatamente $\frac{1}{10}*𝐵$ — ela é uma aproximação. Assim, numericamente 𝐴 fica quase singular, mas não exatamente singular. 

In [12]:
x_num = np.linalg.solve(A,b)
x_num

array([[ 0.1672619 ],
       [ 0.66547619],
       [-0.16607143]])

Como visto acima, métodos diretos podem retornar uma solução numérica porque a matriz representada tem um pequeno desvio que torna o determinante numérico diferente de zero. Entretanto a matriz é extremamente mal condicionada: isso significa que pequenas perturbações (por exemplo erros de arredondamento) podem causar grandes mudanças na solução.


In [25]:
sol_aprox, _, _, _ = np.linalg.lstsq(A,b, rcond=None)
sol_aprox

array([[0.38888889],
       [0.22222222],
       [0.05555556]])

No nosso cálculo com NumPy obtivemos que:

- np.linalg.solve(A,b) retornou uma solução por causa da não-exatidão de 𝐴 em ponto-flutuante.
- O método dos mínimos quadrados utilizando o pseudoinverso de A (np.linalg.lstsq) deu uma solução particular exata (em termos racionais), retornando a solução de norma mínima: $$ 𝑥_𝑝 = [\frac{7}{18}, \frac{2}{9}, \frac{1}{18}]^𝑇$$

As duas soluções diferem por um múltiplo do vetor do núcleo, isto é, ambas são soluções da equação numericamente, e a diferença entre elas é compatível com $$ 𝑡*[−1, 2, −1] $$

Resumindo: o método numérico não "falha" porque o computador trabalha com uma versão levemente perturbada de 𝐴, mas os resultados devem ser tratados com cautela por causa da altíssima sensibilidade.

#### Item D)

O número de condição mede o quão sensível é o sistema a pequenas perturbações nos dados. 

In [26]:
cond_A = np.linalg.cond(A)
cond_A

2.37588029981422e+16

Em ponto-flutuante double (numa máquina com $𝜀_{mach} ≈ 2.22*10^{−16}), a amplificação de erro típica é da ordem de: $$ cond(𝐴)*𝜀_{mach} $$
Nesse caso<br> $$ cond(𝐴)*𝜀_{mach} ≈ 2.3758*10^{16}*2.22*10^{−16} ≈ 5.27 > 1 $$<br>

Ou seja, qualquer erro de arredondamento pode gerar um erro relativo na solução da ordem de unidade — portanto não se pode confiar em dígitos significativos da solução obtida numericamente.<br>
Uma estimação grosseira do número de dígitos decimais corretos é $$ log_{10}(cond(𝐴)) $$<br>
O que dá um número negativo. Em termos práticos isso significa 0 dígitos confiáveis.

Para a matriz exata 𝐴 (matematicamente) o condicionamento é infinito, uma vez que det(𝐴) = 0, o que implica que a matriz não é invertível. Numericamente obtemos um valor muito grande mas finito por causa da representação em ponto flutuante.